In [1]:
# imports
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

10000 - arts and entertainment

12009 - education
12004 - community center
12080 - library
19046 - metro station
15014 - hospital


13032 - cafes, coffee, tea houses
13029 - brewery
13003 - bar
13065 - restaurant

16000 - landmarks and outdoors
18000 - sports and rec

17000 - retail

In [12]:
fs_api_key = os.environ['FOURSQUARE_KEY_2']

location = '49.262487%2C-123.114397'
url = "https://api.foursquare.com/v3/places/search?radius=400&categories=10000%2C12009%2C12004%2C12080%2C13032%2C13029%2C13003%2C13065%2C16000%2C17000%2C18000%2C19046%2C15014&limit=50&ll=" + location

headers = {"accept": "application/json",
           'Authorization': fs_api_key}

response = requests.get(url, headers=headers)

print(response.status_code)

400


In [7]:
response.json()['results'][0]

{'fsq_id': '4aa73d25f964a520704c20e3',
 'categories': [{'id': 17069,
   'name': 'Grocery Store / Supermarket',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
    'suffix': '.png'}},
  {'id': 17071,
   'name': 'Health Food Store',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
    'suffix': '.png'}},
  {'id': 17077,
   'name': 'Meat and Seafood Store',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/foodanddrink_',
    'suffix': '.png'}}],
 'chains': [],
 'distance': 182,
 'geocodes': {'main': {'latitude': 49.263957, 'longitude': -123.115209},
  'roof': {'latitude': 49.263957, 'longitude': -123.115209}},
 'link': '/v3/places/4aa73d25f964a520704c20e3',
 'location': {'address': '510 8th Ave W',
  'country': 'CA',
  'cross_street': 'at Cambie St',
  'formatted_address': '510 8th Ave W (at Cambie St), Vancouver BC V5Z 1C5',
  'locality': 'Vancouver',
  'postcode': 'V5Z 1C5',
  'region': 'BC'},
 'na

In [10]:
len(response.json()['results'])

50

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

active
arts
education
breweries
hospitals
medcenters
metrostations
bars
communitycenters
landmarks
libraries
religiousorgs
restaurants
shopping

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating